In [69]:
import sys
import math
import pickle
import numpy as np

k = 0.1
n = 2
m = 2

sys.setrecursionlimit(1500)

In [70]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [71]:
digitImage = inputDigit()

In [72]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [73]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
#print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000


In [74]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training=np.zeros(shape=(10, int(28/n), int(28/m), 2**(n*m)))

count = 0
for i in range(5000):                  # for each image
    for row in range(int(28/n)):
        for col in range(int(28/m)):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if digitImage[i*28 + n*row + y][m*col + x] != ' ':
                        feature_val += 2**(m*y+x)
            training[int(label[i])][row][col][feature_val] += 1

for i in range(10):
    training[i] = (training[i] + k) / (class_[i][1] + k * 2**(m*n))

#print(training[0][0].shape)
#print(training[0][0])



In [75]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")
answer = np.zeros(1000)

for i in range(1000):                  # for each image
    test_image = np.zeros(shape=(int(28/n), int(28/m)))
    for row in range(int(28/n)):
        for col in range(int(28/m)):      # for each feature
            
            feature_val = 0
            for y in range(n):
                for x in range(m):      # for each pixel in feature (overlapping)
                    # calculate feature_val
                    if test_rough[i*28 + n*row + y][m*col + x] != ' ':
                        feature_val += 2**(m*y+x)
            test_image[row][col] = feature_val
       
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(int(28/n)):
            for col in range(int(28/m)):
                posteriori[class_num] += math.log(training[class_num][row][col][int(test_image[row][col])])
    answer[i] = np.argmax(posteriori)
    #print(posteriori)
    
#print(answer)



In [76]:
testlabels = readRough("digitdata/testlabels")


In [77]:
def confusion_matrix():
    num_each_class = np.zeros(10)
    conf_matrix = np.zeros(shape=(10,10))
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [78]:
conf_matrix = confusion_matrix()
conf_matrix = np.around(conf_matrix, 2)
for row in conf_matrix:
    print(row)
diag_sum = 0
for i in range(10):
    diag_sum += conf_matrix[i][i]
print(diag_sum)

[ 0.93  0.    0.    0.01  0.    0.    0.02  0.    0.03  0.  ]
[ 0.    0.96  0.01  0.    0.01  0.01  0.01  0.    0.    0.  ]
[ 0.01  0.01  0.87  0.02  0.    0.    0.03  0.02  0.04  0.  ]
[ 0.    0.    0.01  0.9   0.    0.04  0.01  0.01  0.01  0.02]
[ 0.    0.01  0.    0.    0.87  0.    0.02  0.01  0.01  0.08]
[ 0.01  0.01  0.    0.09  0.    0.83  0.    0.01  0.03  0.02]
[ 0.01  0.02  0.    0.    0.02  0.08  0.86  0.    0.01  0.  ]
[ 0.    0.05  0.05  0.    0.01  0.01  0.    0.77  0.02  0.09]
[ 0.02  0.01  0.03  0.11  0.01  0.03  0.    0.01  0.78  0.01]
[ 0.01  0.01  0.    0.03  0.08  0.01  0.    0.03  0.02  0.81]
8.58
